In [10]:
import torch
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import time

import datasets.mvtec as mvtec

from feature_extraction import Resnet18Features, concatenateLayers, extractEmbeddingVectors, extractEmbeddingVectorsDataloader, WideResnet50Features, getIndices
from distribution_fitting import getMeanCov
from distance_calculation import calculateScoreMaps
# from padim_utils import saveFeatures

from torchvision.models import wide_resnet50_2, resnet18


In [11]:
use_cuda = torch.cuda.is_available()
# device = torch.device('cuda' if use_cuda else 'cpu')
device = torch.device('cpu')
device

device(type='cpu')

In [12]:
class_name = 'down'
data_path = '../data/aligned_pucks/'
dataset = mvtec.MVTecDataset(data_path, class_name=class_name, is_train=True)
dataloader = DataLoader(dataset, batch_size=32, pin_memory=True)
len(dataset)
# x = next(iter(dataloader))[0][0].unsqueeze(0)
# x.shape

100

In [13]:
features_model = Resnet18Features(device)
# features_model = WideResnet50Features(device)

In [21]:
tic = time.perf_counter()
embedding_vectors = extractEmbeddingVectorsDataloader(features_model, dataloader, device)
print(time.perf_counter()-tic)

Feature extraction: 100%|██████████| 4/4 [00:05<00:00,  1.47s/it]

5.87278752902057


In [22]:
idx = getIndices(100, 448, device)  
# idx = getIndices(550, 1792, device)  
# idx = getIndices(400, 1792, device)  
embedding_vectors = torch.index_select(embedding_vectors, 1, idx)


In [23]:
embedding_vectors.shape

torch.Size([100, 100, 56, 56])

In [24]:
tic = time.perf_counter()
mean, cov, cov_inv = getMeanCov(embedding_vectors, torch.device('cpu'))
print(time.perf_counter()-tic)

Calculating covariance:   0%|          | 0/3136 [00:00<?, ?it/s]

Calculating mean


Calculating covariance: 100%|██████████| 3136/3136 [00:02<00:00, 1153.12it/s]


3.5403875240008347


In [25]:
torch.save(mean, './distributions/down_mean_test.pkl')
# torch.save(cov, './distributions/up_cov_test.pkl')
torch.save(cov_inv, './distributions/down_cov_inv_test.pkl')
